In [2]:
# Define a function that generates samples approximate RGO. The target is defined in Potential class.
import numpy as np
import matplotlib.pyplot as plt
import random
from utils import target_func, estimate_W2_Gaussian
from matplotlib.ticker import MultipleLocator 

In [4]:
# Define a function as the original approximate RGO
def generate_samples(step_size, x_y, f):
    dimension = f.dimension
    ite = 0
    while True:
        samples = np.random.multivariate_normal(mean = x_y, cov = step_size * np.identity(dimension), size = 2)
        # Compute the acceptance probability
        gradient = f.firstOrder(x_y)
        a = f.zeroOrder(samples[0,:])-np.dot(gradient,samples[0,:])
        b = f.zeroOrder(samples[1,:])-np.dot(gradient,samples[1,:])
        # The code works even when rho is inf. One can also take the log transformation
        rho = np.exp(b-a)
        u = np.random.uniform(0,1)
        ite = ite + 1
        if u < rho/2:
            break
    return samples[0,:],ite

# Define a function that estimates the local step size
def estimate_step_size(step_size, tolerance, y, f):
    dimension = f.dimension
    # Compute the desired subexponential parameter
    x_y = f.solve1(y, step_size)
    testFunction = lambda C : np.mean(np.exp(np.abs(Y)**(2/(1+f.alpha))/C))-2
    while True:
        # Generate random samples from a Gaussian distribution: \exp^{-(x-x_y)^2/(2\step_size)}
        samples = np.random.multivariate_normal(mean = x_y, cov = step_size * np.identity(dimension), size = 200)
        Y = np.zeros(100)
        for i in range(100):
            gradient = f.firstOrder(x_y)
            a = f.zeroOrder(samples[i])-np.dot(gradient,samples[i])
            b = f.zeroOrder(samples[i+100])-np.dot(gradient,samples[i+100])
            Y[i] = b-a
        # Estimate the subexponential parameter of Y: find the smallest C>0 such that E[\exp^{\abs(Y)/C}] \leq 2 by binary search for smooth potentials
        # Initialize the interval
        left = 0
        right = dimension**(f.alpha/(f.alpha+1)) # The estimated upper bound of the subexponential parameter
        while testFunction(right)>0:
            left = right
            right = 2*right
        # Initialize the middle point
        mid = (left+right)/2
        # Initialize the value of the function
        f_mid = testFunction(mid)
        while abs(f_mid) > 1e-2:
            if f_mid > 0:
                left = mid
            else:
                right = mid
            mid = (left+right)/2
            f_mid =  testFunction(mid)
        # reduce this 10
        if mid < 1 / ( np.log(6/tolerance) / np.log(2)  * 0.05) :
            break
        else:
            step_size = step_size / 2
            x_y = f.solve1(y, step_size)
    return step_size, x_y

# Define the outer loop of proximal sampler
def proximal_sampler(initial_step_size, num_samples, num_iter, f, fixed):
    dimension = f.dimension

    samples = np.zeros([num_samples,num_iter,dimension])
    Ysamples = np.zeros([num_samples,dimension])
    rejections = np.zeros([num_samples, num_iter])
    
    # Initialize the samples for both fxied and adaptive versions
    samples[:,0,:] = np.random.multivariate_normal(mean =  np.zeros(dimension), cov =  np.identity(dimension), size = num_samples)
    for j in range(num_samples):
        Ysamples[j,:] = np.random.multivariate_normal(mean = samples[j,0,:], cov = initial_step_size * np.identity(dimension), size = 1)
    
    if fixed == True:    
        print(f'fixed sampling')
        for i in range(1,num_iter):
            for j in range(num_samples):
                x_y = f.solve1(Ysamples[j,:], initial_step_size)
                samples[j,i,:],ite = generate_samples(initial_step_size, x_y, f)
                Ysamples[j,:] = np.random.multivariate_normal(mean = samples[j,i,:], cov = initial_step_size * np.identity(dimension), size = 1) 
                rejections[j,i] = ite
            if i % 100 == 0:
                print(f"Steps:{i}")
                if f.times2 > 0:
                    print(f"Averaged optimization steps of the new one: {f.ite2/f.times2}")
                print(f"Averaged rejection steps : {np.mean(rejections[:,i])}")
        
        return samples
    
    if fixed == False:
        print(f'adpative sampling')
        step_sizes = np.zeros([num_samples,num_iter])
        step_sizes[:,0] = initial_step_size
        for i in range(1,num_iter):
            for j in range(num_samples):
                # if i < 100 or (i >= 100 and np.random.uniform(0,1) < 0.001):
                if True:
                    step_size, x_y = estimate_step_size(2*step_sizes[j,i-1], 1e-2, Ysamples[j,:], f)
                else:
                    x_y = f.solve1(Ysamples[j,:], step_sizes[j,i-1])
                    step_size = step_sizes[j,i-1]
                samples[j,i,:],ite = generate_samples(step_size, x_y, f)
                Ysamples[j,:] = np.random.multivariate_normal(mean = samples[j,i,:], cov = step_size * np.identity(dimension), size = 1)  
                step_sizes[j,i] = step_size
                rejections[j,i] = ite  
            
            # statistics for the first sample
            if i % 100 == 0:
                print(f"Steps:{i}")
                print(f"Averaged_step_size:{np.mean(step_sizes[:,i])}")
                if f.times2 > 0:
                    print(f"Averaged optimization steps of the new one: {f.ite2/f.times2}")
                print(f"Averaged rejection steps : {np.mean(rejections[:,i])}")
    return samples, step_sizes
            

In [5]:
def Gaussian_sampler(seed, dimension, numIter, numSamples, initialStep):

    random.seed(seed)
    np.random.seed(seed)

    Target = target_func(dimension, if_simple = True)
    
    # To Do: double check this formula
    hatC = (1+Target.alpha)*(1/Target.alpha)**(Target.alpha/(1+Target.alpha))*(1/np.pi)**(2/(1+Target.alpha))*2**((-1-2*Target.alpha)/(1+Target.alpha))
    min_step_size = hatC/(120*np.log(6/0.01)/np.log(2)*Target.L_alpha*np.sqrt(Target.dimension))
    print(f"min_step_size={min_step_size}")
        
    Xsamples_adaptive ,step_sizes = proximal_sampler(initialStep, numSamples, numIter, f = Target, fixed = False)
    W2_distances = estimate_W2_Gaussian(Xsamples_adaptive,Target.mean, np.identity(dimension)) # the true cov is I_d

    
    return W2_distances, step_sizes

num_iters = 21
trails = 10
overall_results = np.zeros([trails,5,num_iters-1])
overall_sizes = np.zeros([trails,5,num_iters-1])


for index_trail in range(trails):
    sizes = np.zeros([5,num_iters])
    for one_dimen in [2,4,8,16,32]:
        W2_distances, step_sizes = Gaussian_sampler(index_trail,one_dimen,num_iters,50,0.1)
        overall_results[index_trail,int(np.log2(one_dimen)-1),:] =  W2_distances[1:]
        overall_sizes[index_trail, int(np.log2(one_dimen)-1),:] = np.mean(step_sizes[:,1:],axis = 0)

    W2_mean = np.mean(overall_results,axis = 0)
    w2_std = np.std(overall_results,axis = 0)
    plt.figure(figsize = (5,6))
    for dimen_index in range(overall_results.shape[1]):
        y = W2_mean[dimen_index,:]
        error = 2* w2_std[dimen_index,:]
        one_dimen = 2**(dimen_index+1)
        plt.errorbar(list(range(1, num_iters)), y, yerr=error, fmt='o', ecolor='r', 
                     capsize=5, capthick=2, elinewidth=2, label=f'd = {one_dimen}')

    plt.xlabel('Iterations')
    plt.ylabel('Estiamted W2 distance')
    ax = plt.gca() 
    ax.xaxis.set_major_locator(MultipleLocator(4))  
    plt.legend()
    plt.savefig('W2_dimension.pdf')
    plt.close()


    size_mean = np.mean(overall_sizes,axis = 0)
    size_std = np.std(overall_sizes,axis = 0)
    plt.figure(figsize = (5,6))
    for dimen_index in range(overall_sizes.shape[1]):
        y = size_mean[dimen_index,:]
        error = 2* size_std[dimen_index,:]
        one_dimen = 2**(dimen_index+1)
        plt.errorbar(list(range(1, num_iters)), y, yerr=error, fmt='o', ecolor='r', 
                     capsize=5, capthick=2, elinewidth=2, label=f'd = {one_dimen}')
    plt.xlabel('Iterations')
    plt.ylabel('Average step size')
    ax = plt.gca() 
    ax.xaxis.set_major_locator(MultipleLocator(4))  
    plt.legend()
    plt.savefig('step_dimension.pdf')
    plt.close()
    


L=1.0
min_step_size=0.00014371191339995835
adpative sampling
1.0874221352767347e-08
4.1781184370776116e-09
1.862292753055172e-08
L=1.0
min_step_size=0.00010161966850240442
adpative sampling
1.5884472615572737e-08
9.892064183006265e-08
2.170097580261599e-08
L=1.0
min_step_size=7.185595669997918e-05
adpative sampling
3.252135469305443e-08
2.999485520539009e-08
L=1.0
min_step_size=5.080983425120221e-05
adpative sampling
4.7484518521242914e-07
1.3109068035958592e-07
L=1.0
min_step_size=3.592797834998959e-05
adpative sampling
3.578331255939156e-07
2.0571270498869004e-07
4.078514040940647e-07
3.7585902715421966e-07
3.359498615165657e-07
L=1.0
min_step_size=0.00014371191339995835
adpative sampling
9.730191236324793e-09
1.3253306710070836e-09
L=1.0
min_step_size=0.00010161966850240442
adpative sampling
6.263048668280892e-08
L=1.0
min_step_size=7.185595669997918e-05
adpative sampling
3.025141255850048e-07
4.2090859439759185e-08
1.723980127253747e-07
4.0596490497214004e-08
L=1.0
min_step_size=5.

KeyboardInterrupt: 